In [1]:
import torch, torchdata, torchtext
import torch.nn as nn
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

In [2]:
torch.__version__, torchdata.__version__, torchtext.__version__

('2.2.2+cu121', '0.7.1', '0.17.2+cpu')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
SEED = 1234 #change three times
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## 1. load dataset

In [5]:
df = pd.read_csv('data/pre-category/sample100percategoty.csv')
df.head(5)

,questionType,question
0,yes/no,Does this system filter gaillardia?
1,yes/no,Can you get a gas dryer instead of electric
2,yes/no,I have a 2010 maytag centennial washer w101409...
3,yes/no,CAN YOU JUST PLUG IT IN ONCE IT IS SEATED?
4,yes/no,does this replace a thermadore water filter ca...


In [6]:
# convert to lower case
df['question']  =  df['question'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [7]:
# tranform the text question type to integer
df['questionType']=df['questionType'].replace(['yes/no','open-ended'], [0,1])
df.head(5)

,questionType,question
0,0,does this system filter gaillardia?
1,0,can you get a gas dryer instead of electric
2,0,i have a 2010 maytag centennial washer w101409...
3,0,can you just plug it in once it is seated?
4,0,does this replace a thermadore water filter ca...


In [8]:
def data_cleaning(data):
    regex_s = re.sub("\\(.+?\\)|[\r\n|\n\r]|!", "", data)
    fin = " ".join(regex_s.split())
    return fin

In [9]:
df['question'] = df['question'].apply(data_cleaning)

In [10]:
train_df, val_df = train_test_split(df, test_size=0.15,stratify=df['questionType'], random_state=SEED)

In [11]:
train_df, test_df = train_test_split(train_df, test_size=0.15, stratify=train_df['questionType'],random_state=SEED)

In [12]:
train_df['questionType'].value_counts()

questionType
0    759
1    758
Name: count, dtype: int64

In [13]:
val_df['questionType'].value_counts()

questionType
1    158
0    157
Name: count, dtype: int64

In [14]:
test_df['questionType'].value_counts()

questionType
0    134
1    134
Name: count, dtype: int64

In [15]:
test_df['question'][1848]

'can you use this to get the temperature of wine through the bottle?'

## 2.Preprocessing

### Tokenizing

In [16]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
tokens    = tokenizer("What is the best product?")
tokens

['What', 'is', 'the', 'best', 'product', '?']

### Text to integers (numeral)

In [17]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data):
    for data_sample in data:
        yield tokenizer(data_sample) 
        
vocab = build_vocab_from_iterator(yield_tokens(train_df['question']), specials = ['<unk>', '<pad>', '<bos>', '<eos>'])
vocab.set_default_index(vocab["<unk>"])

In [18]:
vocab(['here', 'it', 'is'])

[842, 11, 7]

In [19]:
mapping = vocab.get_itos()
mapping[0]

'<unk>'

In [20]:
len(vocab)

3905

## 3. Data loader

### FastText Embedding

In [21]:
from torchtext.vocab import FastText
fast_vectors = FastText(language='simple')

In [22]:
fast_embedding = fast_vectors.get_vecs_by_tokens(vocab.get_itos()).to(device)

In [23]:
fast_embedding.shape

torch.Size([3905, 300])

In [24]:
class DataWrap(Dataset):

    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __len__(self):
        return len(self.dataframe)
        
    def __getitem__(self, idx):
        return self.dataframe.iloc[idx]

In [25]:
train_df.iloc[0]

questionType                                                    1
question        do these things cover the front and the back o...
Name: 381, dtype: object

In [26]:
train = DataWrap(train_df)
valid = DataWrap(val_df)
test = DataWrap(test_df)

In [27]:
text_pipeline  = lambda x: vocab(tokenizer(x)) #{hello world this is yt} => {'hello', 'world', 'this', 'is', 'yt'} => {4, 88, 11, 22, 6}

In [28]:
text_pipeline("I am currently teaching LSTM")

[0, 66, 2222, 0, 0]

In [29]:
#collate_fn to let each batch has same size
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

pad_idx = vocab['<pad>'] #get index of pad in vocab list
# padding for every sentencce in batch to have same length 

def collate_batch(batch):
    label_list, text_list, length_list = [], [], []
    for (_label, _text) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64) # convert to integer before appending
        text_list.append(processed_text)
        length_list.append(processed_text.size(0))
    return torch.tensor(label_list, dtype=torch.int64), pad_sequence(text_list, padding_value=pad_idx, batch_first=True), torch.tensor(length_list, dtype=torch.int64)

In [30]:
batch_size = 64

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True,  collate_fn=collate_batch) #num_workers to train faster
val_loader   = DataLoader(valid, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader  = DataLoader(test,  batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [31]:
for label, text, length in val_loader:
    break

In [32]:
label.shape #(batch_size, )

torch.Size([64])

In [33]:
text.shape #(batch_size, seq len)

torch.Size([64, 36])

In [34]:
length.shape #(batch_size, )

torch.Size([64])

## 4. Model (biLSTM)

In [35]:
class LSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, num_layers, 
                 bidirectional, dropout, output_dim):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)
        self.lstm      = nn.LSTM(
                            emb_dim,
                            hid_dim,
                            num_layers=num_layers,
                            bidirectional=bidirectional,
                            dropout = dropout,
                            batch_first = True
                        )
        self.fc        = nn.Linear(hid_dim * 2, output_dim) # time 2 b/c bidirectional, output_dim = 4 since there are 4 class
    
    def forward(self, text, text_length):
        #text = [batch_size, seq len]
        embedded = self.embedding(text)
        #text = [batch_size, seq len, emb_dim]
        
        #pack sequence to ignore any padding
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_length.to('cpu'), 
                                                            enforce_sorted=False, batch_first=True)
        
        packed_output, (hn, cn) = self.lstm(packed_embedded)
        #output is basically all the hidden states;  hn is only last hidden state; cn is last cell state
        
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        #output = [batch_size, seq len, hidden_dim * num directions]
        #hn     = [num_layers * num_directions, batch_size,  hid_dim]  #3 layers birectional - hn1f, hn1b, hn2f, hn2b, hn3f, hn3b
        #cn     = [num_layers * num_directions, batch_size,  hid_dim]
        
        hn      = torch.cat((hn[-2, :, :], hn[-1, :, :]), dim = 1) # hn3f, hn3b
        #hn     = [batch_size, hidden_dim * num_directions]
        
        return self.fc(hn)

## 5. Train

In [36]:
def initialize_weight(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight) # normal distribution
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.orthogonal_(param)

In [37]:
def accuracy(preds, y):
    predicted  = torch.max(preds.data, 1)[1] #.data for getting value in tensor
    batch_corr = (predicted == y).sum()
    acc        = batch_corr / len(y)
    return acc

In [38]:
def train(model, loader, optimizer, criterion, loader_length):
    epoch_loss = 0
    epoch_acc  = 0
    model.train()
    
    for i, (label, text, text_length) in enumerate(loader):
        label = label.to(device)
        text  = text.to(device)
        
        predictions = model(text, text_length).squeeze(1)
        
        loss = criterion(predictions, label)
        acc  = accuracy(predictions, label)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc  += acc.item()
        
    return epoch_loss / loader_length, epoch_acc / loader_length

In [39]:
def evaluate(model, loader, criterion, loader_length):
    epoch_loss = 0
    epoch_acc  = 0
    model.eval()
    
    with torch.no_grad():
        for i, (label, text, text_length) in enumerate(loader):
            label = label.to(device)
            text  = text.to(device)
            
            predictions = model(text, text_length).squeeze(1)
            
            loss = criterion(predictions, label)
            acc  = accuracy(predictions, label)
            
            epoch_loss += loss.item()
            epoch_acc  += acc.item()
        
    return epoch_loss / loader_length, epoch_acc / loader_length

### Actual training

In [40]:
train_loader_length = len(list(iter(train_loader)))
val_loader_length   = len(list(iter(val_loader)))
test_loader_length  = len(list(iter(test_loader)))

In [41]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [42]:
dsfasdg

NameError: name 'dsfasdg' is not defined

In [44]:
#experiment tracking
import mlflow
from mlflow.models import infer_signature
import os

# This the dockerized method.
# We build two docker containers, one for python/jupyter and another for mlflow.
# The url `mlflow` is resolved into another container within the same composer.
mlflow.set_tracking_uri("http://mlflow:5000")
# In the dockerized way, the user who runs this code will be `root`.
# The MLflow will also log the run user_id as `root`.
# To change that, we need to set this environ["LOGNAME"] to your name.
os.environ["LOGNAME"] = "noppawee"
#mlflow.create_experiment(name="noppawee-ML-project")  #create if you haven't create
mlflow.set_experiment(experiment_name="question_classification")


<Experiment: artifact_location='mlflow-artifacts:/218882049739984480', creation_time=1713171317283, experiment_id='218882049739984480', last_update_time=1713171317283, lifecycle_stage='active', name='question_classification', tags={}>

In [ ]:
import time
import torch.optim as optim

num_epochs = [5,10]
hid_dims = [128,256,512]
num_layers = [2,4,6]

for num_epoch in num_epochs:
    for hid_dim in hid_dims:
        for num_layer in num_layers:
            input_dim = len(vocab)
            emb_dim   = 300 #fasttext
            output_dim = 2 #2 types of question
            bidirectional = True
            dropout    = 0.5  # dropout between layers

            params={"model":"biLSTM", "num_epochs":num_epoch, "input_dim":input_dim, "hid_dim":hid_dim, "emb_dim":emb_dim, "output_dim":output_dim, "num_layers":num_layer, "dropout":0.5}
            mlflow.start_run(run_name=f"biLSTM100-{params['num_epochs']}-epochs-{params['hid_dim']}-hidden dim-{params['num_layers']}-num layers")
            mlflow.log_params(params)

            print("="*5, f"biLSTM with {params['num_epochs']}-epochs-{params['hid_dim']}-hidden dim-{params['num_layers']}-num layers","="*5)

            model = LSTM(input_dim, emb_dim, hid_dim, num_layer, bidirectional, dropout, output_dim)
            model.apply(initialize_weight)
            model.embedding.weight.data = fast_embedding

            lr = 1e-3
            optimizer = optim.Adam(model.parameters(), lr=lr)
            criterion = nn.CrossEntropyLoss()

            train_losses, train_accs, val_losses, val_accs = [],[],[],[]
            best_valid_loss = float('inf')

            for epoch in range(num_epoch):
                start_time = time.time()
                
                train_loss, train_acc = train(model, train_loader, optimizer, criterion, train_loader_length)
                valid_loss, valid_acc = evaluate(model, val_loader, criterion, val_loader_length)
                
                #for plotting
                train_losses.append(train_loss)
                train_accs.append(train_acc)
                val_losses.append(valid_loss)
                val_accs.append(valid_acc)
                
                end_time = time.time()
                
                epoch_mins, epoch_secs = epoch_time(start_time, end_time)
                mlflow.log_metric(key="train_loss", value=train_loss, step=epoch)
                mlflow.log_metric(key="train_acc", value=train_acc, step=epoch)
                mlflow.log_metric(key="val_loss", value=valid_loss, step=epoch)
                mlflow.log_metric(key="val_acc", value=valid_acc, step=epoch)
                
            
                #early stopping
                if valid_loss < best_valid_loss:
                    best_valid_loss = valid_loss
                    mlflow.pytorch.log_model(model, "model")
                
                print(f'Epoch: {epoch+1} | Time: {epoch_mins}m {epoch_secs}s')
                print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
                print(f'\tVal.  Loss: {valid_loss:.3f} | Val Acc: {valid_acc*100:.2f}%')
            mlflow.log_metric(key="min_val_loss", value=min(val_losses), step=epoch)    
            mlflow.end_run()
        

===== biLSTM with 5-epochs-128-hidden dim-2-num layers =====
Epoch: 1 | Time: 0m 2s
	Train Loss: 0.575 | Train Acc: 71.11%
	Val.  Loss: 0.348 | Val Acc: 82.86%


/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Epoch: 2 | Time: 0m 2s
	Train Loss: 0.350 | Train Acc: 84.73%
	Val.  Loss: 0.315 | Val Acc: 85.31%
Epoch: 3 | Time: 0m 1s
	Train Loss: 0.177 | Train Acc: 93.11%
	Val.  Loss: 0.304 | Val Acc: 84.48%
Epoch: 4 | Time: 0m 1s
	Train Loss: 0.121 | Train Acc: 95.55%
	Val.  Loss: 0.294 | Val Acc: 87.03%
Epoch: 5 | Time: 0m 2s
	Train Loss: 0.063 | Train Acc: 97.85%
	Val.  Loss: 0.334 | Val Acc: 86.27%
===== biLSTM with 5-epochs-128-hidden dim-4-num layers =====
Epoch: 1 | Time: 0m 3s
	Train Loss: 0.519 | Train Acc: 73.04%
	Val.  Loss: 0.710 | Val Acc: 78.07%
Epoch: 2 | Time: 0m 4s
	Train Loss: 0.192 | Train Acc: 92.51%
	Val.  Loss: 0.460 | Val Acc: 81.82%
Epoch: 3 | Time: 0m 4s
	Train Loss: 0.063 | Train Acc: 97.95%
	Val.  Loss: 0.515 | Val Acc: 81.56%
Epoch: 4 | Time: 0m 5s
	Train Loss: 0.031 | Train Acc: 98.96%
	Val.  Loss: 0.648 | Val Acc: 82.78%
Epoch: 5 | Time: 0m 4s
	Train Loss: 0.010 | Train Acc: 99.45%
	Val.  Loss: 0.850 | Val Acc: 82.18%
===== biLSTM with 5-epochs-128-hidden dim-6-num 